# EULER 84: Monopoly Odds
https://projecteuler.net/problem=84


In [1]:
# Set up logging for debug purposes
import Logging
logger = Logging.ConsoleLogger(stderr, Logging.Info)
Logging.global_logger(logger);

## Framework
### Define some types to facilitate simulating the card draws, rolls, and turns of each player

First, the card piles:

In [2]:
import Random: shuffle!
struct CardPile
    height::Integer
    elements::AbstractArray
    
    function CardPile(height::Integer, elements::Array{String})
        while length(elements) < height
            append!(elements, ["nothing"]) # useless cards
        end
        shuffle!(elements)
        new(height, elements)
    end
end

function draw!(pile::CardPile)
    card = popfirst!(pile.elements)
    push!(pile.elements, card)
    @debug "Draw $card"
    card
end;

Dice and players can be defined next. Note that we need to keep track of double-rolls, so the `roll(dice::Dice)` function will return an array of values, not their sum. Naively, `Player`s by default start on Go -- square 1 -- with no history of rolling doubles. However, this leads to a "burn-in" problem when simulating results, as starting on "Go" biases the early turns in favor of early squares. We can get around this by randomly initializing `Player` positions when a `Board` is constructed, since we're interested in the steady-state probability distribution. Thus the initial position will be configurable in the constructor for `Player`.

In [3]:
import Distributions: DiscreteUniform
mutable struct Dice
    sides::Integer
    dist::DiscreteUniform
    number::Integer
    Dice(sides::Integer, number::Integer) = new(sides, DiscreteUniform(1, sides), number)
end

function roll(dice::Dice)
    rand(dice.dist, dice.number)
end

mutable struct Player
    square::String
    position::Integer
    doubles::Array{Bool}
    Player(start::String) = new(start, 1, [false, false])
end

function trackdoubles!(player::Player, roll::Array{T}) where T <: Integer
    player.doubles[1] = player.doubles[end]
    player.doubles[end] = roll[1] == roll[end]
end;

Next we'll set up the `Board` struct, the constructor for which will contain information about the squares and cards used in Monopoly

In [4]:
mutable struct Board
    num_players::Integer
    dice::Dice
    squares::Array{String}
    chance::CardPile
    chest::CardPile
    players::Array{Player}
    
    function Board(num_players, dice)
        squares = split("""GO A1 CC1 A2 T1 R1 B1 CH1 B2 B3
                           JAIL C1 U1 C2 C3 R2 D1 CC2 D2 D3
                           FP E1 CH2 E2 E3 R3 F1 F2 U2 F3
                           G2J G1 G2 CC3 G3 R4 CH3 H1 T2 H2
                           """) .|>  String
        chance = CardPile(16, split("""advance_to_GO go_to_JAIL go_to_C1 go_to_E3
                                       go_to_H2 go_to_R1 go_to_next_R go_to_next_R
                                       go_to_next_U go_back_3""") .|> String)
        chest = CardPile(16, split("advance_to_GO go_to_JAIL") .|> String)
        players = [Player(rand(squares)) for _ in 1:num_players]
        
        new(num_players, dice, squares, chance, chest, players)
    end
end;

With this, we can define the functions that allow `Player`s to interact with the `Board`. 

In [5]:
"""
Set the square string and index for a current player on a board
"""
function setsquare!(player::Player, board::Board, square::String)
    @debug "Move from $(player.square) to $square"
    player.position = findfirst(x->x==square, board.squares)
    player.square = square
    if player.square == "G2J"
        return setsquare!(player, board, "JAIL")
    end
    square
end

"""
Move a Player by an integer number of squares
"""
function move!(player::Player, by::Integer, board::Board)
    #@debug "Move By $by"
    newpos = player.position + by
    if newpos > length(board.squares)
        newpos %= length(board.squares)
    end
    setsquare!(player, board, board.squares[newpos])
end

"""
Move a player to a given square
"""
function move!(player::Player, to::String, board::Board)
    #@debug "Move To $to"
    setsquare!(player, board, to)
end

"""
Determine the final destination square after drawing a CH/CC Card
"""
function handle_card(card::String, player::Player, board::Board)
    
    if occursin("go_to_next_", card)
        substr = split(card, "go_to_next_")[end]
        squares = board.squares[player.position:end]
        append!(squares, board.squares[1:player.position-1])
        @debug "Reordered board: \n $squares"
        return squares[findfirst(x -> occursin(substr, x), squares)]
        
    elseif occursin("go_to_", card)
        return split(card, "go_to_")[end] |> String
        
    elseif occursin("go_back_", card)
        return board.squares[player.position - 3]
        
    elseif occursin("advance_to_", card)
        return "GO"
        
    else
        return player.square
        
    end
end

"""
Player takes a turn on the Board
"""
function taketurn!(player::Player, board::Board)
    thisroll = roll(board.dice)
    @debug "roll $thisroll = $(thisroll |> sum)"
    if thisroll[1] == thisroll[end] && all(player.doubles)
        @debug "Speeding"
        player.doubles = [false, false]
        return move!(player, "JAIL", board)
    end
    trackdoubles!(player, thisroll)
    finish = move!(player, sum(thisroll), board)
    if occursin("CC", finish)
        cardsq = handle_card(draw!(board.chest), player, board)
        if cardsq != finish
            return move!(player, cardsq, board)
        end
    elseif occursin("CH", finish)
        cardsq = handle_card(draw!(board.chance), player, board)
        if cardsq != finish
            return move!(player, cardsq, board)
        end
    end
    return finish
end

"""
Full round of turns for all Players on a Board.

Return the final squares or frequencies for the purposes of tracking frequency.
"""
function fullturn!(board::Board; return_positions=false)
    for (i, player) in enumerate(board.players)
        @debug "Player $i's Turn"
        taketurn!(player, board)
    end
    [p.square for p in board.players]
end;

## Simulation and Results

We'll set up functions to `simulate` a number of games and compute the modal string given a dictionary of counts. Experimentation has shown a large number of simulations is necessary to achieve convergence to the correct probability distribution, so we will exploit Julia's `Threads` module to parallelize the simulations.

In [6]:
import DataStructures: OrderedDict
import Printf: @sprintf
using Plots
plotlyjs()

dice = Dice(6, 2)
board = Board(4, dice)

function modalstring(counts::OrderedDict{String, Int64}, squares::Array{String, 1})
    top3 = sort(collect(counts), by=x->x[2])[end:-1:end-2]
    join(map(z -> @sprintf("%02d", findfirst(x->x==z[1], squares)-1), top3))
end

function simulate(dice::Dice, num_players::Integer, num_games::Integer, num_turns::Integer)
    counts = OrderedDict(zip(board.squares, repeat([0], length(board.squares))))
    
    for i in 1:num_games
        board = Board(num_players, dice)
        for _ in 1:num_turns
            for square in fullturn!(board)
                counts[square] += 1
            end
        end
    end
    normed = OrderedDict()
    for square in keys(counts)
        normed[square] = 100 * counts[square] / sum(values(counts))
    end

    println("Solution: ", modalstring(counts, board.squares))
    bar(normed, size=(900, 300), xticks=((0.5:1:length(board.squares)), board.squares))
    ylabel!("Probability (%)")
end

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-11173999398736247566\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15860382917453145453\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-11173999398736247566\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

simulate (generic function with 1 method)

## Baseline: 6-sided dice

Experimentation shows that the probabilities of "GO", "R1" and "D3" are *very* similar. A large number of simulations is necessary to achieve reliable convergence to the right answer.

Example:

In [7]:
dice = Dice(6, 2)
simulate(dice, 4, 500, 500)

Solution: 102419


<!DOCTYPE html>
 
 
 Plots.jl

20 million total simulated turns yields the correct modal string:

In [11]:
dice = Dice(6, 2)
simulate(dice, 4, 500, 10000)

Solution: 102400


<!DOCTYPE html>
 
 
 Plots.jl

## Problem: 4-sided dice

Thankfully, the four-sided distribution is considerably more differentiated, and not as many simulations are required.

In [9]:
dice = Dice(4, 2)
simulate(dice, 4, 500, 500)

Solution: 101524


<!DOCTYPE html>
 
 
 Plots.jl

# CORRECT ANSWER: 101524